In [1]:
import os
import json
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
from pycocotools import mask
import skimage.measure as measure
from scipy import stats
from tqdm import tqdm

In [2]:
# csv_name order 
csv_names = ['7633.csv', '7416.csv', '7136.csv']

# weight per class for each one
weights = [
    (1,1,1,2,2,1,2,1,1,1,1),
    (1,2,1,1,1,2,1,2,1,1,2),
    (1,1,1,1,1,1,1,1,1,2,1),
]
subs = []
weight = {}
for i, name in enumerate(csv_names):
    subs.append(pd.read_csv('./'+name, index_col=None))
    weight[name] = weights[i]

# Ensemble 코드
print("Start Ensemble...")
# sample_submisson.csv 열기
submission = pd.read_csv('./code/submission/sample_submission.csv', index_col=None)


Start Ensemble...


In [3]:
for ind in tqdm(range(819)):
    dmasks = []
    
    # pseudo_mask = np.zeros((256, 256), dtype=np.int32)
    pseudo_mask = [[[] for __ in range(256)] for __ in range(256)]
    
    # 각 픽셀별로, 가중치를 고려해서 모든 label을 list로 만들기
    file_name = subs[0]['image_id'][ind]
    for name, sub in zip(csv_names,subs):
        dmask = np.fromstring(sub['PredictionString'][ind], dtype=int, sep=' ').reshape(256, -1)
        for i in range(256):
            for j in range(256):
                for k in range(weight[name][dmask[i][j]]):
                    pseudo_mask[i][j].append(dmask[i][j])
    
    # 각 픽셀의 label list를 사용해 최빈값을 하나로 설정하기
    for i in range(256):
        for j in range(256):
            pseudo_mask[i][j] = str(stats.mode(pseudo_mask[i][j])[0].tolist()[0])

    
    predictString = ' '.join(' '.join(e) for e in pseudo_mask)
    submission = submission.append({"image_id" : file_name, "PredictionString" : predictString}, ignore_index=True)
    
# submission.csv로 저장
submission.to_csv("Ensemble_Final_SY.csv", index=False)
print("Done")

100%|██████████| 819/819 [1:31:03<00:00,  6.67s/it]


Done
